# Wariant probabilistyczny:

In [19]:
try:
    import numpy as np
except ImportError:
    print("Sorry, this example requires Numpy installed !")
    raise

from easyAI import TwoPlayerGame
import random
from IPython.display import clear_output


class ConnectFour(TwoPlayerGame):
    """
    The game of Connect Four, as described here:
    http://en.wikipedia.org/wiki/Connect_Four
    """

    def __init__(self, players, board=None):
        self.players = players
        self.board = (
            board
            if (board is not None)
            else (np.array([[0 for i in range(7)] for j in range(6)]))
        )
        self.current_player = 1  # player 1 starts.
        self.start_time = time.time()
        self.move_times = []

    def possible_moves(self):
        return [i for i in range(7) if (self.board[:, i].min() == 0)]

    def make_move(self, column):
        # List where number defines where the ball will fall (-1 - left column; 0 - center column; 1 - right column)
        actions = [-1, 0, 1]
        action = actions[random.randint(0, len(actions) - 1)]
        # Checking if column index is different than 0 or 6 (edges)
        if action == -1 and column != 0 or action == 1 and column != 6:
            column += action
        line = np.argmin(self.board[:, column] != 0)
        self.board[line, column] = self.current_player

    def show(self):
        end_time = time.time()
        move_time = end_time - self.start_time
        if(move_time != 0):
            self.move_times.append(move_time)
            print("CZAS RUCHU: " + str(move_time))
            self.start_time = time.time()
        # print(
        #     "\n"
        #     + "\n".join(
        #         ["0 1 2 3 4 5 6", 13 * "-"]
        #         + [
        #             " ".join([[".", "O", "X"][self.board[5 - j][i]] for i in range(7)])
        #             for j in range(6)
        #         ]
        #     )
        # )
        pass

    def lose(self):
        return find_four(self.board, self.opponent_index)

    def is_over(self):
        return (self.board.min() > 0) or self.lose()

    def scoring(self):
        return -100 if self.lose() else 0


def find_four(board, current_player):
    """
    Returns True iff the player has connected  4 (or more)
    This is much faster if written in C or Cython
    """
    for pos, direction in POS_DIR:
        streak = 0
        while (0 <= pos[0] <= 5) and (0 <= pos[1] <= 6):
            if board[pos[0], pos[1]] == current_player:
                streak += 1
                if streak == 4:
                    return True
            else:
                streak = 0
            pos = pos + direction
    return False


POS_DIR = np.array(
    [[[i, 0], [0, 1]] for i in range(6)]
    + [[[0, i], [1, 0]] for i in range(7)]
    + [[[i, 0], [1, 1]] for i in range(1, 3)]
    + [[[0, i], [1, 1]] for i in range(4)]
    + [[[i, 6], [1, -1]] for i in range(1, 3)]
    + [[[0, i], [1, -1]] for i in range(3, 7)]
)



if __name__ == "__main__":
    # LET'S PLAY !

    from easyAI import AI_Player, Negamax, SSS
    
    player1wins = 0
    player2wins = 0
    
    #for i in range(20):

    ai_algo_neg = Negamax(7)
    ai_algo_sss = Negamax(7)
    game = ConnectFour([AI_Player(ai_algo_neg), AI_Player(ai_algo_sss)])
    game.play()
    if game.lose():
        print("Player %d wins." % (game.opponent_index))
        print("Sredni czas ruchu: " + str(mean(game.move_times)))
        if game.opponent_index == 1:
            player1wins += 1     
        else: player2wins +=1    
    else:
        print("Looks like we have a draw.")
#     clear_output() 
     
#     print("wygrane gracza 1 : " + str(player1wins))
#     print("wygrane gracza 2 : " + str(player2wins))


Move #1: player 1 plays 0 :
CZAS RUCHU: 1.8072476387023926

Move #2: player 2 plays 0 :
CZAS RUCHU: 1.8063409328460693

Move #3: player 1 plays 0 :
CZAS RUCHU: 2.4959969520568848

Move #4: player 2 plays 0 :
CZAS RUCHU: 1.6156411170959473

Move #5: player 1 plays 0 :
CZAS RUCHU: 1.7094581127166748

Move #6: player 2 plays 0 :
CZAS RUCHU: 1.268460750579834

Move #7: player 1 plays 0 :
CZAS RUCHU: 1.4361014366149902

Move #8: player 2 plays 0 :
CZAS RUCHU: 0.9923171997070312

Move #9: player 1 plays 0 :
CZAS RUCHU: 0.6989016532897949

Move #10: player 2 plays 0 :
CZAS RUCHU: 1.0413599014282227

Move #11: player 1 plays 2 :
CZAS RUCHU: 0.8387753963470459

Move #12: player 2 plays 2 :
CZAS RUCHU: 0.5654399394989014

Move #13: player 1 plays 2 :
CZAS RUCHU: 0.4890594482421875

Move #14: player 2 plays 3 :
CZAS RUCHU: 0.6476480960845947

Move #15: player 1 plays 2 :
CZAS RUCHU: 0.8849420547485352

Move #16: player 2 plays 2 :
CZAS RUCHU: 0.40133094787597656

Move #17: player 1 plays 2 :
CZA


# Wariant deterministyczny:

In [14]:
try:
    import numpy as np
except ImportError:
    print("Sorry, this example requires Numpy installed !")
    raise

from easyAI import TwoPlayerGame
import time
from statistics import mean

class ConnectFour(TwoPlayerGame):
    """
    The game of Connect Four, as described here:
    http://en.wikipedia.org/wiki/Connect_Four
    """

    def __init__(self, players, board=None):
        self.players = players
        self.board = (
            board
            if (board is not None)
            else (np.array([[0 for i in range(7)] for j in range(6)]))
        )
        self.current_player = 1  # player 1 starts.
        self.start_time = time.time()
        self.move_times = []

    def possible_moves(self):
        return [i for i in range(7) if (self.board[:, i].min() == 0)]

    def make_move(self, column):
        line = np.argmin(self.board[:, column] != 0)
        self.board[line, column] = self.current_player

    def show(self):
        end_time = time.time()
        move_time = end_time - self.start_time
        if(move_time != 0):
            self.move_times.append(move_time)
            print("CZAS RUCHU: " + str(move_time))
            self.start_time = time.time()
        print(
            "\n"
            + "\n".join(
                ["0 1 2 3 4 5 6", 13 * "-"]
                + [
                    " ".join([[".", "O", "X"][self.board[5 - j][i]] for i in range(7)])
                    for j in range(6)
                ]
            )
        )

    def lose(self):
        return find_four(self.board, self.opponent_index)

    def is_over(self):
        return (self.board.min() > 0) or self.lose()

    def scoring(self):
        return -100 if self.lose() else 0


def find_four(board, current_player):
    """
    Returns True iff the player has connected  4 (or more)
    This is much faster if written in C or Cython
    """
    for pos, direction in POS_DIR:
        streak = 0
        while (0 <= pos[0] <= 5) and (0 <= pos[1] <= 6):
            if board[pos[0], pos[1]] == current_player:
                streak += 1
                if streak == 4:
                    return True
            else:
                streak = 0
            pos = pos + direction
    return False


POS_DIR = np.array(
    [[[i, 0], [0, 1]] for i in range(6)]
    + [[[0, i], [1, 0]] for i in range(7)]
    + [[[i, 0], [1, 1]] for i in range(1, 3)]
    + [[[0, i], [1, 1]] for i in range(4)]
    + [[[i, 6], [1, -1]] for i in range(1, 3)]
    + [[[0, i], [1, -1]] for i in range(3, 7)]
)

if __name__ == "__main__":
    # LET'S PLAY !

    from easyAI import AI_Player, Negamax, SSS

    ai_algo_neg = Negamax(7)
    ai_algo_sss = Negamax(7)
    game = ConnectFour([AI_Player(ai_algo_neg), AI_Player(ai_algo_sss)])
    game.play()
    if game.lose():
        print("Player %d wins." % (game.opponent_index))
        print("Sredni czas ruchu: " + str(mean(game.move_times)))
    else:
        print("Looks like we have a draw.")


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .

Move #1: player 1 plays 0 :
CZAS RUCHU: 2.3811123371124268

0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O . . . . . .

Move #2: player 2 plays 0 :
CZAS RUCHU: 0.8471982479095459

0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
X . . . . . .
O . . . . . .

Move #3: player 1 plays 0 :
CZAS RUCHU: 1.145552396774292

0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
O . . . . . .
X . . . . . .
O . . . . . .

Move #4: player 2 plays 0 :
CZAS RUCHU: 0.7815825939178467

0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
X . . . . . .
O . . . . . .
X . . . . . .
O . . . . . .

Move #5: player 1 plays 0 :
CZAS RUCHU: 2.0820984840393066

0 1 2 3 4 5 6
-------------
. . . . . . .
O . . . . . .
X . . . . . .
O . . . . . .
X . . . . . .
O . . . . . .

Move #6: player 2 play

# Opis zaimplementowanego wariantu Connect Four
Zaimplementowany przez nas wariant rozszerza grę o możliwość przesunięcia spadającej z góry kulki odpowiednio o jedną kolumnę w lewo lub w prawo. Implementacja bazuje na liscie *actions* definiującej poszczególne akcje:
- -1 - Przesunięcie spadającego elementu o jedną kolumnę w lewo
- 0  - Brak przesunięcia 
- 1  - Przesunięcie o jedną kolumnę w prawo

Wylosowanie każdej akcji jest tak samo prawdopodobne i wynosi 33.(3)%.
### Działanie
Przy każdym wywołaniu funkcji make_move, losowany jest indeks tablicy *actions* z zakresu (0, len(actionos) - 1) a następnie dokonywane jest sprawdzenie i wartość odpowiadająca temu indeksowi jest dodawana do wybranej przez algorytm kolumny. Sprawdzenie, to instrukcje warunkowe, które zapobiegają sytuacjom, w których algorytm wybiera dla obiektu skrajną kolumnę a my próbujemy przesunąć obiekt poza planszę.
### Wyniki
##### Zaczyna gracz nr 1 dla max głebokości: 5
|                  | probabilistyczny | deterministyczny |   
|------------------|------------------|------------------|
| wygrane gracza 1 |        11        |         0        |
| wygrane gracza 2 |         9        |        20        |
|      remisy      |         0        |         0        |

##### Zaczyna gracz nr 2 dla max głebokości: 5

|                  | probabilistyczny | deterministyczny |   
|------------------|------------------|------------------|
| wygrane gracza 1 |         9        |        20        |
| wygrane gracza 2 |        11        |         0        |
|      remisy      |         0        |         0        |

##### Zaczyna gracz nr 1 dla max głebokości: 7

|                  | probabilistyczny | deterministyczny |   
|------------------|------------------|------------------|
| wygrane gracza 1 |        10        |         0        |
| wygrane gracza 2 |        10        |        20        |
|      remisy      |         0        |         0        |

##### Zaczyna gracz nr 2 dla max głebokości: 7

|                  | probabilistyczny | deterministyczny |   
|------------------|------------------|------------------|
| wygrane gracza 1 |        10        |         0        |
| wygrane gracza 2 |        10        |        20        |
|      remisy      |         0        |         0        |
### Wnioski
##### Wersja determistyczna
Po przeanalizowaniu wyników, zauważyliśmy, że podstawowa (deterministyczna) werjsa gry jest powtarzalna i zawsze przegrywa w niej gracz który zaczyna. Gracze grają w dość głupi sposób. Rozgrywke rozpoczynają od lewego dolnego rogu planszy i wypełniają kolejne kolumny od lewej do prawej. Gra zawsze składa się z 38 ruchów i przebiega w identyczny sposób, niezależnie od doboru parametrów algorytmu Negamax. 
##### Wersja probabilistyczna
Wersja probabilistyczna diametralnie zmienia wyniki rozgrywki. Staje się ona bardziej wyrównana i zmienia się w zależności od maksymalnej głębokości algorytmu.  
- Przy maksymalnej głębokości 5 i początku rozgrywki jako gracz pierwszy, wygrywa on 11 na 20 gier, co daje 55% skuteczności.
- Przy maksymalnej głębokości 5 i początku rozgrywki jako gracz drugi, wygrywa on 11 na 20 gier, co daje 55% skuteczności.  

Pokazuje to, że przy głębokości 5, gracz rozpoczynający rozgrywkę, wciąż ma delikatnie większe szanse na wygraną, jednak jest to rozgrywka wyrównana w przeciwieństwie do wariantu deterministycznego.  
- Przy maksymalnej głębokości 7 i początku rozgrywki jako gracz pierwszy, wygrywa on 10 na 20 gier, co daje 50% skuteczności.
- Przy maksymalnej głębokości 7 i początku rozgrywki jako gracz drugi, wygrywa on 10 na 20 gier, co daje 50% skuteczności.   
Można z tego wywnioskować, że zwiększenie głębokości wyrównuje szanse dla obu graczy.

### Średnie czasy ruchów dla różnych wariantów
##### Deterministyczny wariant z głębokością 5
Sredni czas ruchu: 0.08887432123485364
##### Deterministyczny wariant z głębokością 7
Sredni czas ruchu: 0.5177107923909238
##### Probabilistyczny wariant z głębokością 5
Sredni czas ruchu: 0.19188524547376132
##### Probabilistyczny wariant z głębokością 7
Sredni czas ruchu: 0.7993360634507805